**DTS - Complete Streets** *Right-of-Way Widths for Planned Street Improvements*



# 06 - Street Typology Assignment

**Author:** rmangan

___

**Purpose:**

Utilzie all previously compiled attributes in modal composite 05, Complete Streets Design Manual, and direct input from project stakeholders to auto-assign Complete Street Types to street segments. To be used as a starting point for manual review by project team.


**This script performs the following functions:**

1. Calculate Urban and Agricultural segment length % values

2. Assign Typologies

3. (optional utility functions) Reset all typologies to NULL for reprocessing


**Global Assumptions and Notes:**
1. None. See individual sections for assmuptions and notes related to each function

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - geo-processing

In [ ]:
# import modules
import arcpy
import os

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

zoning = r"\\dc1vs01\GISProj\B\Base\HI\Honolulu_City_County\zoning\CCH_zoning.gdb\zoning_LUO_20210505"

In [ ]:
#Add Required Fields

#add CS type field
arcpy.AddField_management(modal_composite_05a,field_name="CS_type",field_type="TEXT",field_alias = "Complete Streets - Type", field_length = 30)


## 01 - Calculate Urban and Agricultural segment length % values
---

Assign Complete Street typologies based on attributes in modal composite dataset, complete streets design manual, and direct input from project stakeholders.

**This section performs the following functions:**

1. Calculate % of each line segment, by length, that intersects with Urban land use zones.
2. Calculate % of each line segment, by length, that intersects with Agricultural land use zones.

**Assumptions and Notes:**
1. Land Use Zoning from: City & County of Honolulu Open Geospatial Data - Zoning [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Urban Land use zones
   1. **Aloha** - State Jurisdiction: Aloha Tower Project (Admin. by Aloha Tower Development Corp.)
   2. **AMX-2**
   3. **AMX-3**
   4. **B-1** - B-1 Neighborhood Business District
   5. **B-2** - B-2 Community Business District
   6. **BMX-3** - BMX-3 Community Business Mixed Use District
   7. **BMX-4** - BMX-4 Central Business Mixed Use District
   8. **Kak** - State Jurisdiction: Kakaako Community Development District (Admin. by HCDA)
   9. **MU** - Mixed Use Precinct (Kakaako Special Design District)
   10. **PU** - Public Use Precinct (Kakaako Special Design District)
   11. **Pub** - Public Precinct (Waikiki Special District)
   12. **ResMix** - Resort Mixed Use Precinct (Waikiki Special District)
   13. **Resort** - Resort District

3. Agricultural Land use zones:
   1. **AG-1**
   2. **AG-2**

In [ ]:
#Compute Urban zoning line percentage metric

#select appropriate zoning categories, using the following ZONE_CLASS values

# Aloha - State Jurisdiction: Aloha Tower Project (Admin. by Aloha Tower Development Corp.)
# AMX-2
# AMX-3
# B-1 - B-1 Neighborhood Business District
# B-2 - B-2 Community Business District
# BMX-3 - BMX-3 Community Business Mixed Use District
# BMX-4 - BMX-4 Central Business Mixed Use District
# Kak - State Jurisdiction: Kakaako Community Development District (Admin. by HCDA)
# MU - Mixed Use Precinct (Kakaako Special Design District)
# PU - Public Use Precinct (Kakaako Special Design District)
# Pub - Public Precinct (Waikiki Special District)
# ResMix - Resort Mixed Use Precinct (Waikiki Special District)
# Resort - Resort District



print("create urban zoning layer...")
urban_zoning = arcpy.management.MakeFeatureLayer(zoning, "urban_zoning", where_clause='"ZONE_CLASS" IN(\'Aloha\', \'AMX-2\', \'AMX-3\', \'B-1\', \'B-2\', \'BMX-3\', \'BMX-4\', \'Kak\', \'MU\', \'PU\', \'Pub\', \'ResMix\', \'Resort\')' )

print("dissolve zoning...")
dissolve_output = os.path.join(scratch_gdb_path, "urban_zoning_dissolve")
urban_zoning_dissolve = arcpy.management.Dissolve(urban_zoning, dissolve_output)

# intersect modal comp w/ selected zoning features
print("intersect zoning w/ modal_comp...")
intersect_output = os.path.join(scratch_gdb_path, "modal_zoning_intersect")
modal_zoning_intersect = arcpy.analysis.Intersect([modal_composite, urban_zoning_dissolve], intersect_output)

#add length field if it doesnt already exist
print("add length_urban field...")
list_fields = arcpy.ListFields(modal_zoning_intersect)
field_names = [i.name for i in list_fields]

if "length_urban" in field_names:
    print("length_urban field already exists")
else:
    print("adding length_urban field...")
    arcpy.AddField_management(modal_zoning_intersect,field_name="length_urban",field_type="FLOAT")
    print("done")

    
#calc field from shape.length with update cursor
print("calc length_urban field from shape.length...")
with arcpy.da.UpdateCursor(modal_zoning_intersect,["Shape_Length","length_urban"]) as cursor:
    for row in cursor:
        try:
            row[1] = row[0]        
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
print("done")

#run frequency on output and sum length fields to normalize data by SegmentID
print("summarize intersect table by SegmentID...")
freq_output = os.path.join(scratch_gdb_path, "freq_output")
freq = arcpy.analysis.Frequency(modal_zoning_intersect, freq_output, "SEGMENTID", "length_urban")


#join intersect freq result back to modal comp
print("Join field to modal_comp on SegmentID...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = freq
join_table_field = "SEGMENTID"
join_fields = ["length_urban"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#add new field for urban length % if it doesn't already exist
print("add new field for urban length percentage...")
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

if "length_urban_percent" in field_names:
    print("length_urban_percent field already exists")
else:
    print("adding length_urban_percent field...")
    arcpy.AddField_management(modal_composite,field_name="length_urban_percent",field_type="FLOAT", field_alias = "Urban % by length")
    print("done")

    
#calculate line percentages based on length differences

#calc field from shape.length with update cursor
print("calc urban length percent field...")
with arcpy.da.UpdateCursor(modal_composite,["length_urban","Shape_Length","length_urban_percent","SEGMENTID"]) as cursor:
    for row in cursor:
        try:
            if row[0] is not None:
                row[2] = row[0]/row[1]  
                print("SegmentID: {0}, Length: {1}, Urban Length: {2}, Urban Percent: {3}".format(row[3], row[1], row[0], row[2]))
                
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
                       
            
##Compute Ag zoning line percentage metric

#select appropriate zoning categories, using the following ZONE_CLASS values
# AG-1 -
# AG-2 -


print("create ag zoning layer...")
ag_zoning = arcpy.management.MakeFeatureLayer(zoning, "ag_zoning", where_clause='"ZONE_CLASS" IN (\'AG-1\', \'AG-2\')' )

print("dissolve zoning...")
ag_dissolve_output = os.path.join(scratch_gdb_path, "ag_zoning_dissolve")

ag_zoning_dissolve = arcpy.management.Dissolve(ag_zoning, ag_dissolve_output)

# intersect modal comp w/ selected zoning features
print("intersect ag zoning w/ modal_comp...")
ag_intersect_output = os.path.join(scratch_gdb_path, "modal_ag_zoning_intersect")
modal_ag_zoning_intersect = arcpy.analysis.Intersect([modal_composite, ag_zoning_dissolve], ag_intersect_output)


#add length field if it doesnt already exist
print("add length_ag field...")
list_fields = arcpy.ListFields(modal_ag_zoning_intersect)
field_names = [i.name for i in list_fields]

if "length_ag" in field_names:
    print("length_ag field already exists")
else:
    print("adding length_ag field...")
    arcpy.AddField_management(modal_ag_zoning_intersect,field_name="length_ag",field_type="FLOAT",field_alias = "Ag length")
    print("done")


#calc field from shape.length with update cursor
print("calc length_ag field from shape.length...")
with arcpy.da.UpdateCursor(modal_ag_zoning_intersect,["Shape_Length","length_ag"]) as cursor:
    for row in cursor:
        try:
            row[1] = row[0]        
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
print("done")


#run frequency on output and sum length fields to normalize data by SegmentID
print("summarize intersect table by SegmentID...")
ag_freq_output = os.path.join(scratch_gdb_path, "ag_freq_output")
ag_freq = arcpy.analysis.Frequency(modal_ag_zoning_intersect, ag_freq_output, "SEGMENTID", "length_ag")


#join intersect freq result back to modal comp
print("Join field to modal_comp on SegmentID...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = ag_freq
join_table_field = "SEGMENTID"
join_fields = ["length_ag"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#add new field for urban length % if it doesn't already exist
print("add new field for Ag length percentage...")
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

if "length_ag_percent" in field_names:
    print("length_ag_percent field already exists")
else:
    print("adding length_ag_percent field...")
    arcpy.AddField_management(modal_composite,field_name="length_ag_percent",field_type="FLOAT", field_alias = "Ag % by length")
    print("done")

    
#calculate line percentages based on length differences

#calc field from shape.length with update cursor
print("calc ag length percent field...")
with arcpy.da.UpdateCursor(modal_composite,["length_ag","Shape_Length","length_ag_percent","SEGMENTID"]) as cursor:
    for row in cursor:
        try:
            if row[0] is not None:
                row[2] = row[0]/row[1]  
                print("SegmentID: {0}, Length: {1}, Ag Length: {2}, Ag Percent: {3}".format(row[3], row[1], row[0], row[2]))
                
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
            
            
print("processing complete")


## 02 - Assign Typologies
---

Assign Complete Street typologies based on attributes in modal composite dataset, complete streets design manual, and direct input from project stakeholders.

**This section performs the following functions:**

1. Calculate % of each line segment, by length, that intersects with Urban land use zones.
2. Calculate % of each line segment, by length, that intersects with Agricultural land use zones.

**Assumptions and Notes:**
1. Land Use Zoning from: City & County of Honolulu Open Geospatial Data - Zoning [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Urban Land use zones
   1. **Aloha** - State Jurisdiction: Aloha Tower Project (Admin. by Aloha Tower Development Corp.)
   2. **AMX-2**
   3. **AMX-3**
   4. **B-1** - B-1 Neighborhood Business District
   5. **B-2** - B-2 Community Business District
   6. **BMX-3** - BMX-3 Community Business Mixed Use District
   7. **BMX-4** - BMX-4 Central Business Mixed Use District
   8. **Kak** - State Jurisdiction: Kakaako Community Development District (Admin. by HCDA)
   9. **MU** - Mixed Use Precinct (Kakaako Special Design District)
   10. **PU** - Public Use Precinct (Kakaako Special Design District)
   11. **Pub** - Public Precinct (Waikiki Special District)
   12. **ResMix** - Resort Mixed Use Precinct (Waikiki Special District)
   13. **Resort** - Resort District

3. Agricultural Land use zones:
   1. **AG-1**
   2. **AG-2**

In [ ]:
#Set CSDM analysis fields and filters

#fields to expose to update cursor
cs_fields = ["AB_LANE",               #0
             "BA_LANE",               #1
             "lanes_assumed",         #2
             "AB_SPEED",              #3
             "BA_SPEED",              #4
             "mean_row_2020",         #5
             "median_type",           #6
             "median_width",          #7
             "TOT_FLOW_D",            #8
             "CS_type",               #9
             "SEGMENTID",             #10
             "FULLNAME",              #11
             "length_urban_percent" , #12     
             "length_ag_percent",     #13
             "aadt",                  #14
             "SPD_LIMIT"]             #15

count_blvd = 0
count_ave = 0
count_main = 0
count_street = 0
count_res_street = 0
count_lane = 0
count_mall = 0
count_rural = 0
count_scenic = 0
count_error = 0

print("Setting Street Types....")
count = 0
with arcpy.da.UpdateCursor(modal_composite, cs_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #Set Boulevards
            if (((row[0] is not None) and (row[1] is not None) and row[0] + row[1] >= 4) and #lanes >=4                            
               ((row[15] is not None) and row[15] > 35) or row[6] == "R"): #Speed Limit >35
                
                count_blvd += 1
                row[9] = "Boulevard"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                      
            #Set Main Streets
            elif ((row[15] is not None) and (row[15] <= 35) and #Speed Limit <=35
                  (row[14] is not None and row[14] >= 10000) and #AADT > 10000
                  (row[12] is not None and row[12] > .5)): #Urban Percent > 50%
                
                count_main += 1
                row[9] = "Main Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                
            #Set Avenues            
            elif ((row[15] is not None) and (row[15] <= 35) and #Speed Limit <=35
                  (row[14] is not None and row[14] >= 10000) and #AADT > 10000
                  ((row[12] is not None and row[12] <= .5) or row[12] is None)): #Urban Percent is None or less than 50%
                
                count_ave += 1
                row[9] = "Avenue"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                    
            #Set Malls - hardcoded based on Segment Ids for Dukes Ln, Fort St Mall, S. Hotel St, Kekaulike St.
            elif (row[10] in (18914, 18967, 18984, 
                              19033, 19090, 19114, 
                              19148, 19152, 19233, 
                              19234, 19271, 19272, 
                              19306, 19319, 19333, 
                              19357, 19358, 19411, 
                              19456)):
                count_mall += 1
                row[9] = "Mall"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            #Set Lanes & Alleys - ROW < 30 and has 2 lanes max and does not have TransCAD listing
            elif ((row[5] is not None and row[5] < 30) and #width < 30
                  (((row[0] is not None) and (row[1] is not None) and (row[0] + row[1] <=2)) or row[0] is None)): #2 lanes max or TransCAD is null 
                count_lane += 1
                row[9] = "Lane/Alley"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            #Set Rural Roads - Rural % > 50 and does not have TransCAD or AADT data
            elif (row[13] is not None and row[13] > .5 and (row[0] is None) and (row[14] is None)):
                count_rural += 1
                row[9] = "Rural Road"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                
            #Set Scenic Byway's - hardcoded
            elif (row[10] in (18589,18648,23714,
                              33873,16814,33532,
                              33533,33534,33535,
                              33536,33537,33538)):
                count_scenic += 1
                row[9] = "Scenic Byway"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                  
            #Set Street        
            elif ((row[15] is not None) and (row[15] <= 25) and #Speed Limit <=25
                  ((row[0] is not None) and (row[1] is not None) and row[0] + row[1] <= 4) and #lanes <= 4   
                  (row[14] is not None) and row[14] >= 4000 and row[14] <= 10000): #AADT > 10000 
                
                count_major_st += 1
                row[9] = "Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                    
                    
            #Set Everything else to Residential Street
            else:
                count_st += 1
                row[9] = "Residential Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            count_error += 1
            
print("Type - Boulevard: {}".format(count_blvd))
print("Type - Avenue: {}".format(count_ave))
print("Type - Main: {}".format(count_main))
print("Type - Mall: {}".format(count_mall))
print("Type - Lane: {}".format(count_lane))
print("Type - Rural: {}".format(count_rural))
print("Type - Scenic Byway: {}".format(count_scenic))
print("Type - Street: {}".format(count_street))
print("Type - Residential Street: {}".format(count_res_street))
print("Types set for {} records".format((count_blvd +
                                         count_ave +
                                         count_main +
                                         count_mall +
                                         count_lane +
                                         count_rural +
                                         count_scenic +
                                         count_street +
                                         count_res_street)))

print("I am Error: {}x".format(count_error))

print("Typology Assignments complete.")


## 03 - Utility Functions

In [ ]:
#reset all CS Street Types to Null, use during testing and debugging

with arcpy.da.UpdateCursor(modal_composite,"CS_type") as cursor:
    for row in cursor:
        #print("row")
        try:
            row[0] = None
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("All CS Types reset to None")
          